In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
from keras.layers import Dense, Input,Flatten, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from keras import backend as K
import keras
import matplotlib.pyplot as plt
import utm
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from keras.utils import to_categorical

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()


def distance_loss(y_pred, y_true):
    return K.sqrt(K.mean(K.sum(K.square(y_pred-y_true), axis=-1)))

def median_absolute_deviation(y_pred, y_true):
    deviation = np.abs(y_pred-y_true)
    return np.mean(deviation, axis=0)


data = pd.read_csv('trainfix.csv')
label = pd.read_csv('label1.csv')['label']
X_temp = [
    np.concatenate([mr[4:9],mr[9:14],mr[14:19],mr[19:24],mr[24:29],
               mr[29:34]]) for mr in data.values
]
X_temp = np.array(X_temp)
y_temp = data[['Latitude','Longitude']].values

X = []
y = []
for t in range(72):
    temp = X_temp[data['TrajID'] == t]
    temp_y = y_temp[data['TrajID'] == t]
    for i in range(len(temp)-5):
        X.append(temp[i:i+6])
        y.append(temp_y[i:i+6])
X = np.array(X)
y = np.array(y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=33)

Using TensorFlow backend.


In [2]:
#执行这里会覆盖X_test 出最终结果
data_test = pd.read_csv('testfix.csv')
TrajID=data_test['TrajID'].values
print(TrajID.shape)
X_test = [
    np.concatenate([mr[4:9],mr[9:14],mr[14:19],mr[19:24],mr[24:29],
               mr[29:34]]) for mr in data_test.values
]
X_test = np.array(X_test)
X_final = []
traj_id=[]
for t in range(72):
    temp = X_test[(data_test['TrajID'] == t)]
    for i in range(len(temp)-5):
        X_final.append(temp[i:i+6])
        traj_id.append(t)
X_test = np.array(X_final)
history = LossHistory()

(2470,)


In [4]:
def build_model_ae():
    m = Sequential()
    m.add(LSTM(64, return_sequences=True, input_shape=(6,30)))
    m.add(LSTM(32, return_sequences=True))
    m.add(TimeDistributed(Dense(30)))
    r = Adam(lr=0.001)
    m.compile(optimizer=r, loss='mse', metrics=['mse'])
    return m

def build_model_lstm():
    m = Sequential()
    m.add(LSTM(32, return_sequences=True, input_shape=(6, 64)))
    m.add(LSTM(64, return_sequences=True))
    m.add(TimeDistributed(Dense(1024, activation='relu')))
    m.add(TimeDistributed(Dense(1024, activation='relu')))
    m.add(TimeDistributed(Dense(891, activation='softmax')))
    r = Adam(lr=0.001)
    m.compile(optimizer=r, loss='categorical_crossentropy', metrics=['accuracy'])
    return m

model1 = build_model_ae()
model1.fit(X, X, epochs=80, batch_size=32)
# model1.fit(X_train, X_train, validation_data=(X_test, X_test), epochs=80, batch_size=32)
model1 = Model(inputs=model1.inputs, outputs=model1.layers[0].output)
model1.summary()
X_train_vec = model1.predict(X)
X_test_vec = model1.predict(X_test)

Epoch 1/80
9687/9687 [==============================] - 9s 898us/step - loss: 0.0386 - mean_squared_error: 0.0386
Epoch 2/80
9687/9687 [==============================] - 9s 909us/step - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 3/80
9687/9687 [==============================] - 9s 922us/step - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 4/80
9687/9687 [==============================] - 9s 912us/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 5/80
9687/9687 [==============================] - 9s 904us/step - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 6/80
9687/9687 [==============================] - 8s 870us/step - loss: 0.0024 - mean_squared_error: 0.0024
Epoch 7/80
9687/9687 [==============================] - 9s 883us/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 8/80
9687/9687 [==============================] - 8s 862us/step - loss: 0.0016 - mean_squared_error: 0.0016
Epoch 9/80
9687/9687 [==============================] - 9s 904us/step - loss: 0.0014 - m

9687/9687 [==============================] - 9s 902us/step - loss: 8.9881e-06 - mean_squared_error: 8.9881e-06
Epoch 69/80
9687/9687 [==============================] - 8s 858us/step - loss: 1.1410e-05 - mean_squared_error: 1.1410e-05
Epoch 70/80
9687/9687 [==============================] - 8s 833us/step - loss: 9.0982e-06 - mean_squared_error: 9.0982e-06
Epoch 71/80
9687/9687 [==============================] - 9s 903us/step - loss: 9.0551e-06 - mean_squared_error: 9.0551e-06
Epoch 72/80
9687/9687 [==============================] - 9s 907us/step - loss: 9.0482e-06 - mean_squared_error: 9.0482e-06
Epoch 73/80
9687/9687 [==============================] - 9s 911us/step - loss: 8.7634e-06 - mean_squared_error: 8.7634e-06
Epoch 74/80
9687/9687 [==============================] - 9s 914us/step - loss: 9.5940e-06 - mean_squared_error: 9.5940e-06
Epoch 75/80
9687/9687 [==============================] - 9s 887us/step - loss: 8.9792e-06 - mean_squared_error: 8.9792e-06
Epoch 76/80
9687/9687 [=====

In [5]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_train_vec, y, test_size=0.3, random_state=33)

grid=pd.read_csv('grid1.csv')[['x','y']].values
Y_train=[]
for seq in y:
    poses=[]
    for mr in seq:
        u1, u2, _, _ = utm.from_latlon(mr[0], mr[1])
        test = np.array([u1, u2])
        poses.append(np.argmin(np.sum(np.square(test - grid), axis=1)))
    Y_train.append(poses)

Y_train=to_categorical(np.array(Y_train),891)
# Y_test=[]
# y_test_utm=[]
# for seq in y_test:
#     poses=[]
#     utms=[]
#     for mr in seq:
#         u1, u2, _, _ = utm.from_latlon(mr[0], mr[1])
#         test = np.array([u1, u2])
#         utms.append([u1, u2])
#         poses.append(np.argmin(np.sum(np.square(test - grid), axis=1)))
#     y_test_utm.append(utms)
#     Y_test.append(poses)

# Y_test=to_categorical(np.array(Y_test),891)

In [6]:
model2 = build_model_lstm()
model2.fit(X_train_vec, Y_train,epochs=100, batch_size=64)
model2.summary()
y_result = [0]*len(data_test)
y_result_count = [0]*len(data_test)

Epoch 1/100
9687/9687 [==============================] - 5s 530us/step - loss: 5.3043 - acc: 0.0457
Epoch 2/100
9687/9687 [==============================] - 5s 475us/step - loss: 4.6446 - acc: 0.0744
Epoch 3/100
9687/9687 [==============================] - 5s 475us/step - loss: 4.2033 - acc: 0.1056
Epoch 4/100
9687/9687 [==============================] - 4s 457us/step - loss: 3.8877 - acc: 0.1231
Epoch 5/100
9687/9687 [==============================] - 4s 441us/step - loss: 3.6996 - acc: 0.1376
Epoch 6/100
9687/9687 [==============================] - 4s 459us/step - loss: 3.5343 - acc: 0.1595
Epoch 7/100
9687/9687 [==============================] - 4s 447us/step - loss: 3.3489 - acc: 0.1779
Epoch 8/100
9687/9687 [==============================] - 4s 459us/step - loss: 3.1666 - acc: 0.1984
Epoch 9/100
9687/9687 [==============================] - 4s 450us/step - loss: 2.9569 - acc: 0.2314
Epoch 10/100
9687/9687 [==============================] - 4s 462us/step - loss: 2.7140 - acc: 0.2752

9687/9687 [==============================] - 5s 477us/step - loss: 0.1164 - acc: 0.9593
Epoch 83/100
9687/9687 [==============================] - 5s 472us/step - loss: 0.1263 - acc: 0.9566
Epoch 84/100
9687/9687 [==============================] - 5s 480us/step - loss: 0.1128 - acc: 0.9612
Epoch 85/100
9687/9687 [==============================] - 4s 457us/step - loss: 0.1207 - acc: 0.9584
Epoch 86/100
9687/9687 [==============================] - 4s 446us/step - loss: 0.1006 - acc: 0.9649
Epoch 87/100
9687/9687 [==============================] - 5s 470us/step - loss: 0.1037 - acc: 0.9649
Epoch 88/100
9687/9687 [==============================] - 4s 428us/step - loss: 0.0998 - acc: 0.9652
Epoch 89/100
9687/9687 [==============================] - 4s 443us/step - loss: 0.1062 - acc: 0.9636
Epoch 90/100
9687/9687 [==============================] - 5s 465us/step - loss: 0.1095 - acc: 0.9626
Epoch 91/100
9687/9687 [==============================] - 5s 474us/step - loss: 0.0907 - acc: 0.9688
Epo

In [ ]:
import math
def Dist_Error(pred,real):
    errors=np.abs(pred-real)
    result=[math.sqrt(math.pow(i[0],2)+math.pow(i[1],2))for i in errors]
    return np.array(result)
testRe=model2.predict_classes(X_test)
zone_number = 51
zone_letter = 'R'
testRe=testRe.reshape(-1)
y_test_utm=np.array(y_test_utm).reshape(-1,2)

testFi=[[grid[i][0], grid[i][1]] for i in testRe]

In [ ]:
errors=Dist_Error(np.array(testFi),np.array(y_test_utm))
errors=np.array(errors)
print(np.median(errors))
print(np.mean(errors))
print(np.sort(errors)[int(len(errors)*0.9)])

In [ ]:
x = np.arange(0,np.max(errors))
error_y = [len(errors[errors < i])/len(errors) for i in x]
plt.plot(x, error_y)

In [7]:
y_pred = model2.predict_classes(X_test_vec)

In [8]:
import math
def traj_merge(traj_id,trajs,code=0):
    maxleng=len(trajs[0])
    last=-1
    result=[]
    for i in range(len(traj_id)):
        if traj_id[i]!=last:
            result+=trajs[i]
            last=traj_id[i]
        else:
            result.append(trajs[i][maxleng-1])
    return result

In [9]:
y_p = traj_merge(traj_id,y_pred.tolist())

In [10]:
grid = pd.read_csv('grid1.csv').values
zone_number = 51
zone_letter = 'R'
y_final = [[utm.to_latlon(grid[i][1], grid[i][2], zone_letter=zone_letter, zone_number=zone_number)[1],
           utm.to_latlon(grid[i][1], grid[i][2], zone_letter=zone_letter, zone_number=zone_number)[0]] for i in y_p]
y_final = np.array(y_final)
df_pred = pd.DataFrame(data={'Longitude':y_final[:,0], 'Latitude':y_final[:,1]})
df_pred.to_csv('pred6.csv', index=False)
print('success')

success
